In [1]:
from google.colab import drive 
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [2]:
# 5. user_play_df : user_id / game_id / play_time

from multiprocessing import Pool
from tqdm import tqdm
import time

import json, urllib.request
import pandas as pd
import sys

import random


def user_playtime(user_id):
    
    # http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key=97879FFFBE63A4B04CEA9B49437BFCFF&steamid=76561198850951810&format=json
    key = '97879FFFBE63A4B04CEA9B49437BFCFF' # key 0 (lockcept)
    # key = 'D52BB50BAF141918B06724C5736E5995'  # key 1 (se)
    # key = 'CB6523DDDB44C5F11B663987471AE06D' # key 2 (al)
    # key = '977C26430C6B47600E8DCC949687FB81' # key 3 (ji)
    # key = '5A61D607E5DC1204B1A5C5480C95C8EA' # key 4 (hs)
    json_url = 'http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key=%s&steamid=%s&format=json'

    game_list = []

    while True:
        try:
            # time.sleep(1 + 1 * random.random())
            # print (json_url % (key, user_id))

            with urllib.request.urlopen(json_url % (key, user_id)) as url:
                data = json.loads(url.read())
            break
        except Exception as e:
            exc_type, exc_value, exc_traceback = sys.exc_info()
            traceback_details = {
                'filename': exc_traceback.tb_frame.f_code.co_filename,
                'lineno': exc_traceback.tb_lineno,
                'name': exc_traceback.tb_frame.f_code.co_name,
                'type': exc_type.__name__,
                'message': str(e)
            }
            print(user_id, " ", traceback_details)

    if not data['response']:
        return []

    game_count = data['response']['game_count']
    for i in range(game_count):
        now_dict = data['response']['games'][i]
        game_list.append({'user_id': user_id, 'game_id': now_dict['appid'], 'play_time': now_dict['playtime_forever']})

    return game_list


if __name__ == '__main__':

    dir = '/gdrive/My Drive/Colab Notebooks/data/'

    split_num = 100

    user_list_total = pd.read_csv(dir + "user_list.csv", encoding='utf-8', header=0)['user_id']
    total_len = len(user_list_total)
    for i in range(54, 57):
        index_s = (total_len * i) // split_num
        index_e = (total_len * (i + 1)) // split_num
        print(index_s, "~", index_e)
        user_list = user_list_total[index_s: index_e]

        pool = Pool(processes=8)
        start_time = time.time()
        list_user_playtime = []
        with tqdm(total=len(user_list)) as pbar:
            for result in tqdm(pool.imap_unordered(user_playtime, user_list)):
                pbar.update()
                list_user_playtime.append(result)
        pool.close()
        pool.join()

        list_df_user_playtime = sum(list_user_playtime, [])

        df_user_playtime = pd.DataFrame(list_df_user_playtime, columns=['user_id', 'game_id', 'play_time'])

        log_txt = str(i) + ": " + str(len(user_list)) + "\t" + str(len(list_df_user_playtime)) + "\t" + str(len(df_user_playtime['user_id'].unique()))
        log_txt = log_txt + "\t%s seconds\n" % (time.time() - start_time)
        f = open(dir + "playtime/user_playtime_log.txt", 'a')
        f.write(log_txt)
        f.close()

        file_name = dir + "playtime/user_playtime_%03d.csv"

        df_user_playtime.to_csv(file_name % i, encoding='utf-8', index=False)

        print("--- %s seconds ---" % (time.time() - start_time))

152689 ~ 155626


 25%|██▍       | 734/2937 [00:22<01:04, 34.31it/s]
734it [00:22, 34.28it/s]
 56%|█████▌    | 1649/2937 [00:49<00:42, 30.21it/s]
1649it [00:49, 29.15it/s]
100%|█████████▉| 2934/2937 [01:27<00:00, 38.59it/s]
2934it [01:27, 38.61it/s]
100%|██████████| 2937/2937 [01:27<00:00, 33.58it/s]


--- 96.57385158538818 seconds ---
155626 ~ 158562


100%|█████████▉| 2933/2936 [01:26<00:00, 35.43it/s]
2933it [01:26, 35.46it/s]
100%|██████████| 2936/2936 [01:26<00:00, 33.89it/s]


--- 90.99882531166077 seconds ---
158562 ~ 161498


100%|█████████▉| 2934/2936 [01:21<00:00, 32.84it/s]
2934it [01:21, 33.05it/s]
100%|██████████| 2936/2936 [01:21<00:00, 35.92it/s]


--- 83.02240085601807 seconds ---


In [0]:
# 6. user_vote_df : user_id / game_id / vote

from multiprocessing import Pool
from tqdm import tqdm
import time

from bs4 import BeautifulSoup
import requests
import pandas as pd
import sys

def user_vote(profile):
    
    url_review = 'https://steamcommunity.com/profiles/' + str(profile) + '/reviews'
    vote_list = []
    game_id_list = []

    while True:
        try:
            # time.sleep (0.5)
            raw = requests.get(url_review)
            doc = BeautifulSoup(raw.text, 'html.parser')
            
            reviews = doc.select("div.giantNumber").text
            vote_list_raw = doc.select("div.title a")
            for v in vote_list_raw:
                vote = v.text
                if vote == 'Recommended':
                    vote_list.append(1)
                else:
                    vote_list.append(-1)

            game_id_list_raw = doc.select("div.leftcol a")
            for g in game_id_list_raw:
                link = g.attrs['href'].split('/')
                game_id_list.append(link[-1])
            break
        except Exception as e:
            exc_type, exc_value, exc_traceback = sys.exc_info()
            traceback_details = {
                'filename': exc_traceback.tb_frame.f_code.co_filename,
                'lineno': exc_traceback.tb_lineno,
                'name': exc_traceback.tb_frame.f_code.co_name,
                'type': exc_type.__name__,
                'message': str(e)
            }
            print(profile, " ", traceback_details)

    list = []

    for j in range(len(vote_list)):
        list.append({'user_id': str(profile), 'game_id': game_id_list[j], 'vote': vote_list[j]})
    return list

if __name__ == '__main__' :

    dir = '/content/drive/My Drive/Colab Notebooks/data'

    split_num = 100
    user_list_total = pd.read_csv(dir + "/user_list.csv", encoding='utf-8', header=0)['user_id']
    total_len = len(user_list_total)

    for i in range(50,100): 
        index_s = (total_len*i)//split_num
        index_e = (total_len*(i+1))//split_num
        print (index_s, "~", index_e)
        user_list = user_list_total[index_s : index_e]

        pool = Pool(processes = 64)

        start_time = time.time()
        list_user_vote = []
        with tqdm(total=len(user_list)) as pbar:
            for result in tqdm(pool.imap_unordered(user_vote, user_list)):
                pbar.update()
                list_user_vote.append (result)
        pool.close()
        pool.join()

        list_df_user_vote = sum (list_user_vote, [])
        df_user_vote = pd.DataFrame (list_df_user_vote, columns= ['user_id', 'game_id','vote'])

        log_txt = str(i) +": " + str(len(user_list)) + "\t"+ str(len(list_df_user_vote))+ "\t"+ str(len(df_user_vote['user_id'].unique()))
        log_txt = log_txt + "\t%s seconds\n" % (time.time() - start_time)
        f = open(dir+"/vote/user_vote_log.txt", 'a')
        f.write(log_txt)
        f.close()

        file_name = dir + "/vote/user_vote_%03d.csv"

        df_user_vote.to_csv(file_name % i, encoding='utf-8', index=False)

        print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# !pip install selenium
# !apt-get update # to update ubuntu to correctly run apt install
# !apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
import time
import pandas as pd

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('./chromedriver', options=chrome_options)

user_id = 76561198992018077
url = "https://steamcommunity.com/profiles/%d/games/?tab=all" % user_id
driver.get(url)
time.sleep(0.1)

con = driver.find_elements_by_css_selector("div.gameListRow")

game_id_list = []
play_time_list = []

for c in con:
    game_id = c.get_attribute('id').split('_')[1]
    p = c.find_element_by_css_selector("h5.hours_played").text.split(' ')[0]
    
    game_id_list.append(game_id)
    if p == '':
        play_time_list.append(0)
    else:
        play_time_list.append(p)
        
df = pd.DataFrame(data = {'game_id':game_id_list, 'play_time':play_time_list})
df['user_id'] = user_id